# AFSK Data Transmission

In this notebook we will implement a fully-functional Binary Frequency Shift Keying data transmission system working in the audio band (hence the "A" in AFSK). From Wikipedia:

> Frequency-shift keying (FSK) is a frequency modulation scheme in which digital information is transmitted through discrete frequency changes of a carrier signal. [...] The simplest FSK is binary FSK (BFSK) [where] a pair of discrete frequencies transmit binary (0s and 1s) information. With this scheme, the 1 is called the mark frequency and the 0 is called the space frequency. 

In our implementation the two frequencies are in the audible range and therefore this transmission scheme can be used to send information from a loudspeaker to a microphone if desired. 

In [ ]:
import numpy as np
import scipy.signal as sp
import matplotlib.pyplot as plt
%matplotlib inline

import IPython

In [ ]:
plt.rcParams["figure.figsize"] = (14,4)

## Some utility functions

In [ ]:
def save_audio(audio, filename):
    from scipy.io import wavfile
    resc = np.array(32600 * audio, dtype=np.int16)
    wavfile.write(filename, 32000, resc)

In [ ]:
def plot_mag_spec(data, Fs=2*np.pi, pts=None, maxf=None, minf=None, one_sided=True):
    pts = len(data) if pts is None else pts
    w = Fs * (np.arange(0, pts) / pts - 0.5)
    X = np.abs(np.fft.fftshift(np.fft.fft(data, pts)))
    
    maxf = Fs / 2 if maxf is None else maxf
    minf = -Fs / 2 if minf is None else minf
    if one_sided and minf < 0:
        minf = 0
    start = max(0, int(pts * (Fs / 2 + minf) / Fs))
    stop = min(pts, int(pts * (Fs / 2 + maxf) / Fs))
    
    #fig = plt.figure(figsize=(14,4))
    #fig.add_subplot(111).plot(w[start:stop], X[start:stop]);    
    plt.plot(w[start:stop], X[start:stop]);

## Transmission protocol

### Data transmission

During data transmission, every zero in the input bitstram will result in a signal segment containing a sinusoid at the so-called "space" frequency while ones will be encoded by a segment containing a sinusoid at the so-called "mark" frequency. These "segments" are called **symbol intervals** and they are of equal length.

In order to minimize the effective bandwidth of the transmitted signal, the transition between segments at different frequencies is performed by preserving the phase of the signal and jump discontinuities are thus avoided. An example is as follows:

In [ ]:
N = 500
x = np.zeros(N)
phase = 0
for n in range(0, N):
    x[n] = np.sin(phase)
    phase = phase + 2 * np.pi / (20 if n & 0x80 else 40)
plt.plot(x);

The duration of each symbol interval is determined by the desired data rate in terms of bits per second (BPS). Obviously, there is an upper limit on the achievable rate, since decoding of the transmitted signal demands that at least a few full cycles of either sinusoidal oscillations are produced for each input bit.

Transmission reliability is also dependent on the bit rate, with lower rates providing redundancy and thus more robustness to noise. A detailed analysis of the bit error rate for FSK signaling is beyond the scope of this notebook but many references can be found online.

### Synchronization

One of the major difficulties in the design of a receiver is designing a synchronization mechanism that "locks" the receiver to the timing reference of the transmitter. In this simple implementation we won't attempt to design any sophisticated timing recovery system but we will still need to provide the receiver with a reference "start time" that can be used to determine the boundaries between successive symbol intervals in the incoming signal into symbol segments as accurately as possible. 

This is achieved by means of a synchronization preamble; the transmitter will:
 1. send a pilot sinusoid at a pre-defined frequency to alert the receiver of an incoming data stream
 1. introduce a phase reversal in the pilot at time $t_0$
 1. start sending data at time $t_0 + t_d$
 
The value of $t_d$ is known at the receiver and so, if the phase reversal is correctly detected, receiver synchronization can be achieved.

## The Transmitter

The implementation of the transmitter is straightforward:


In [ ]:
class AFSK:
    # transmission protocol:
    #  - at least 500ms of pilot tone
    #  - phase reversal at waveform peak
    #  - 200ms pilot
    #  - 200ms silence
    #  - data
    # by convention, a zero is SPACE, a 1 is MARK
    MARK_FREQ = 1200
    SPACE_FREQ = 2200
    PILOT_FREQ = 400
    BPS = 100     # bits per second
    PILOT_HEAD = 0.5 # seconds
    PILOT_TAIL = 0.4 # seconds
    GAP_LEN = 0.2 # seconds        

In [ ]:
class Transmitter(AFSK):
    def __init__(self, Fs=32000):
        # sampling frequency
        self.Fs = Fs
        # samples per symbol
        self.spb = int(Fs / self.BPS) 
        
    def transmit(self, text):
        # convert string to bitstream
        return self.transmit_bits(''.join(format(ord(i), '08b') for i in text))
        
    def transmit_random(self, num_bits):
        bitstream = []
        # use a simple feedforward scrambler with H(z) = 1 + z^{-18} + z^{-23} fed with ones
        buf, ix = np.zeros(23, dtype=np.uint8), 0
        for n in range(0, num_bits):
            buf[ix] = 0x01 ^ buf[(ix + 17) % 23] ^ buf[(ix + 22) % 23]
            bitstream.append(buf[ix])
            ix = (ix + 1) % 23
        return self.transmit_bits(''.join(str(i) for i in bitstream))
        
    def transmit_bits(self, bits, num_times=1):
        # prepare output array
        pilot_head_len = int(self.Fs * self.PILOT_HEAD)
        pilot_tail_len = int(self.Fs * self.PILOT_TAIL)
        preamble_len = pilot_head_len + pilot_tail_len + int(self.Fs * self.GAP_LEN)
        y = np.zeros(preamble_len + len(bits) * self.spb * num_times + self.spb)
        
        # send pilot head; pilot head must end with phase 0 or pi
        pilot_frequency = 2 * np.pi * self.PILOT_FREQ / self.Fs
        y[:pilot_head_len] = np.flipud(np.cos(pilot_frequency * np.arange(0, pilot_head_len)))
        y[pilot_head_len:pilot_head_len+pilot_tail_len] = -np.cos(pilot_frequency * np.arange(0, pilot_tail_len))
        
        
        # send data after gap
        ix = preamble_len
        # SPACE and MARK phase increments
        phase_inc = 2 * np.pi * np.array([self.SPACE_FREQ, self.MARK_FREQ]) / self.Fs
        phase = 0
        for t in range(0, num_times):
            for k in range(0, len(bits)):
                for n in range(0, self.spb):
                    y[ix] = np.cos(phase)
                    ix += 1
                    phase += phase_inc[int(bits[k])]
                    while phase > 2 * np.pi:
                        phase -= 2* np.pi
        return y  

### The audio data stream

Because the AFSK trasmitter operates in the audio band, we can listen to the resulting signal; note the phase reversal in the initial pilot tone and the data stream starting after the gap.

In [ ]:
Fs = 32000
y = Transmitter(Fs).transmit_random(1000)
IPython.display.Audio(y, rate=Fs)

Here is the shape of the signal; note how the envelope is flat, which makes FSK the preferred transmission system for channels exhibiting nonlinear characteristics.

In [ ]:
plt.plot(y);

We can zoom in on the phase reversal in the pilot tone: 

In [ ]:
plt.plot(y[15600:16400]);

and on a portion of the data stream:

In [ ]:
plt.plot(y[101000:103000]);

The spectrum of the AFSK signal shows that most of the energy is concentrated around the two carrier frequencies:

In [ ]:
plot_mag_spec(y[50000:], Fs=Fs, maxf=5000)

## The Receiver

### Basic detection mechanism

The simplest demodulation mechanism for BPSK is provided by an _incoherent_ receiver, that is, by a receiver that does not try to lock to the phase of the incoming signal. Instead, we use two narrowband filters centered on the mark and space pilot frequencies; with this
 * the power at each filter's output is accumulated over time
 * at the end of each symbol interval the two energy values are compared to decide whether a zero or a one was transmitted
 * the power accumulators are reset

If the main disturbance introduced by the channel is noise, and if the power spectral density of the noise is the same over the two carrier frequencies (which is the case if the noise is additive and white), incoherent detection works quite well.

A nonflat channel response, on the other hand, would affect the amplitude of the two carriers differently and would have to be compensated for via an equalizer. We will not address this problem here.

In [ ]:
class Filter:
    # generic class to implement the transfer function H(z) = B(z)/A(z)
    # filter computes also the output power via a leaky integrator
    def __init__(self, b, a, leak=0.95):
        assert len(a) == len(b), 'must have same number of coefficients in filter -- zero pad if necessary'
        self.a = a
        self.b = b
        self.N = len(a)
        self.y = [0] * self.N
        self.x = [0] * self.N
        self.ix = 0
        self.value = 0
        self.leak = leak
        self.power = 0

    def compute(self, x):
        self.x[self.ix] = x
        self.value = self.b[0] * x
        for n in range(1, self.N):
            k = (self.ix + self.N - n ) % self.N
            self.value += self.b[n] * self.x[k] - self.a[n] * self.y[k]
        self.y[self.ix] = self.value
        self.ix = (self.ix + 1) % self.N
        self.power = self.leak * self.power + (1 - self.leak) * self.value * self.value
        return self.value
    
    
class EllipticBandpass(Filter):
    def __init__(self, order, center, bw, Fs, ripple=2.5, att=50, leak=0.95):
        W = np.array([center - bw/2, center + bw/2]) / (Fs/2)
        super().__init__(*sp.ellip(order, ripple, att, W, btype='bandpass'), leak=leak)

### Channel impairments

As is always the case in communications systems, receiver design is much more complicated than transmitter design since the receiver needs to try and compensate for the impairments in the signal introduced by the channel.

In this simple implementation we will address only two problems:
 * reversing the attenuation introduced by the channel via an Automatic Gain Control (AGC) module
 * establishing a reliable timing reference

#### AGC

The following AGC circuit computes a smooth estimate of the current input power via a slow integrator and adapts the gain factor in order for the input signal to have the prescribed target power when scaled by said gain factor.

In [ ]:
class AGC:
    # automatic gain control cricuit
    def __init__(self, target=1, alpha=0.01, lmb=0.995):
        self.max_gain = 50
        self.target = target
        self.alpha = alpha
        self.lmb = lmb
        self.input_power = 1
        self.gain = 1

    def set_speed(self, alpha):
        self.alpha = alpha

    def update(self, x):
        self.input_power = self.lmb * self.input_power + (1 - self.lmb) * x * x
        self.gain += self.alpha * (self.target - self.input_power)
        self.gain = max(0, min(self.gain, self.max_gain))

#### Timing reference

As we said, the transmitter sends a preamble consisting of a sinusoid in which a timing reference is encoded by an abrupt phase shift of $\pi$ radians. The location in time of this phase reversal becomes the reference start time for the receiver; we assume that the internal clocks of transmitter and receivers are working at the same frequency and therefore establishing a reference time once is all that's needed for decoding.

Initially, the receiver monitors the output power of a bandpass filter centered over the pilot's frequency; when this power exceeds a given threshold, pilot detection is achieved and the receiver looks for a phase reversal. To do so, the input is filtered with  a notch filter centered on the pilot's frequency; the notch "kills" the pilot when the latter is stable but any discontinuity in the pilot results in a large negative value for the output.

In [ ]:
class Notch:
    # simple complex-zero-pair notch filter
    def __init__(self, center, Fs):
        self.c = -2 * np.cos(2 * np.pi * center / Fs)
        self.x1 = 0
        self.x2 = 0
        self.value = 0
        
    def compute(self, x):
        self.value = x + self.c * self.x1 + self.x2
        self.x2 = self.x1
        self.x1 = x
        return self.value        

Here is a simple demonstration of the process (which, in "real life", will have to deal with potential false positives)

In [ ]:
pilot_filter = EllipticBandpass(3, AFSK().PILOT_FREQ, 700, Fs, leak=0.995)
timing_detector = Notch(AFSK().PILOT_FREQ, Fs)

pilot = Transmitter(Fs).transmit_random(0)
pp = np.zeros(len(pilot))
pr = np.zeros(len(pilot))
for n in range(0, len(pilot)):
    pilot_filter.compute(pilot[n])
    pr[n] = timing_detector.compute(pilot[n])
    pp[n] = pilot_filter.power
plt.plot(pp[15700:16200], label='pilot power')
plt.plot(pr[15700:16200], label='power of notched pilot')
plt.legend();

### Implementation details

The receiver's main loop is structured as a state machine: every input sample is first scaled by the gain computed by the AGC module and fed back to the AGC module for gain updates; then, the sample is dispatched to the appropriate code section according to the internal state:
  * ``WAIT_PILOT``: waiting for the appearance of a pilot at the nominal frequency
  * ``WAIT_SYNC``: detect potential phase reversals in the pilot until the pilot ends. Upon detection a timer is started with the known delay between phase reversal and beginning of data
  * ``WAIT_DATA``: waiting for the countdown to reach zero after the pilot ends
  * ``ONLINE``: decoding incoming data stream


In [ ]:
class Receiver(AFSK):
    # state machine
    WAIT_PILOT = 0
    WAIT_SYNC = 10
    WAIT_DATA = 30
    ONLINE = 40
    
    monitor = {}
    m_ix = 0

    def __init__(self):
        # received operates at fixed rate
        self.Fs = 32000
        # samples per bit
        self.spb = self.Fs / self.BPS

        # leaky integrator to compute signal power
        self.signal = Filter([1], [1], leak=0.995)
        
        # Automatic Gain Control
        self.agc = AGC(1, 0.005)
        self.agc_wait_len = int(self.Fs * 0.3)

        # pilot and timing
        self.timing_detector = Notch(self.PILOT_FREQ, self.Fs)
        self.reference_power = 0
        self.timing_reference = [-0.5, 0] # threshold, time

        # bandpass filters for PILOT, MARK and SPACE
        self.pilot = EllipticBandpass(3, self.PILOT_FREQ, 700, self.Fs, leak=0.995)
        self.mark = EllipticBandpass(3, self.MARK_FREQ, 900, self.Fs)
        self.space = EllipticBandpass(3, self.SPACE_FREQ, 900, self.Fs)

        # collect incoming bits into bytes
        self.decision = 0
        self.octet = ''

        # state machine
        self.state = self.WAIT_PILOT
        self.timer = self.agc_wait_len # time to wait after detecting a pilot for the AGC to stabilize
        self.ix = 0
        
    def receive(self, audio, plot_internals=False):
        self.monitor = {
            'signal': np.zeros(len(audio)),
            'gain': np.zeros(len(audio)),
            'power': np.zeros(len(audio)),
            'pilot': np.zeros(len(audio)),
            'decision': np.zeros(len(audio)),
            'timing': np.zeros(len(audio)),
        }
        self.m_ix = 0
        for x in audio:
            self.receive_sample(x)
   
        if plot_internals:
            for key in ['signal', 'power', 'pilot', 'gain']: #
                plt.plot(self.monitor[key], label=key)
            plt.legend(loc="upper center")
        
        return self.monitor
        
    def receive_sample(self, x):
        x *= self.agc.gain  # apply gain control and update AGC
        self.agc.update(x)        
        
        self.signal.compute(x)  # compute signal power

        if self.state == self.WAIT_PILOT:
            self.pilot.compute(x)
            if self.pilot.power > 0.25 * self.signal.power:
                self.timing_detector.compute(x)
                self.timer -= 1
                if self.timer <= 0:
                    # pilot detected; wait for phase reversal
                    self.reference_power = 0.1 * self.pilot.power
                    self.timing_reference = [-np.sqrt(self.pilot.power), self.ix]
                    # turn off AGC until data arrives
                    self.agc.set_speed(0.00)
                    self.state = self.WAIT_SYNC
                    print(f'pilot detected ({self.ix})')
            else:
                self.timer = self.agc_wait_len

        elif self.state == self.WAIT_SYNC:
            self.pilot.compute(x)
            pr = self.timing_detector.compute(x) 
            # detect phase reversal and keep track of location
            if pr < self.timing_reference[0]:
                self.timing_reference = [pr, self.ix]
                print(f'timing reference: {self.timing_reference}')
            if self.pilot.power <  self.reference_power:
                self.pilot.power = 0
                # number of samples before data starts:
                self.timer = int((self.PILOT_TAIL + self.GAP_LEN) * self.Fs) - (self.ix - self.timing_reference[1])
                self.state = self.WAIT_DATA
                print(f'pilot end detected at {self.ix}')
                
        elif self.state == self.WAIT_DATA:
            self.pilot.compute(x)
            # start running mark and space narrowband filters
            self.mark.compute(x)
            self.space.compute(x)
            self.timer -= 1
            if self.timer <= 0:
                # slow down AGC
                self.agc.gain *= 0.5
                self.agc.set_speed(0.002)
                self.decision = 0
                self.octet = ''
                self.timer = self.spb
                self.state = self.ONLINE
                print(f'data starts ({self.ix})\n')

        elif self.state == self.ONLINE:
            self.mark.compute(x)
            self.space.compute(x)
            # accumulate power from bandpass filters and decide on bit value at the end of the symbol period
            self.decision += abs(self.mark.value)
            self.decision -= abs(self.space.value)
            self.timer -= 1
            if self.timer <= 0:
                self.octet += '1' if (self.decision > 0) else '0'
                if len(self.octet) == 8:
                    print(chr(int(self.octet, 2)), end='', flush=True)
                    self.octet = ''
                self.decision = 0
                self.timer += self.spb
                if self.signal.power < self.reference_power:
                    self.pilot.power = 0
                    self.state = self.WAIT_PILOT
                    print(f'\n\ndata signal lost, waiting for new pilot tone')

        self.ix += 1
        
        self.monitor['signal'][self.m_ix] = x
        self.monitor['gain'][self.m_ix] = self.agc.gain
        self.monitor['power'][self.m_ix] = self.signal.power
        self.monitor['pilot'][self.m_ix] = self.pilot.power
        self.monitor['decision'][self.m_ix] = self.decision
        self.monitor['timing'][self.m_ix] = self.timing_detector.value
        self.m_ix += 1

### Testing back-to-back

In [ ]:
y = Transmitter(Fs).transmit("hello, and welcome to this DSP class. I hope you will find signal processing interesting and fun!")
Receiver().receive(y, plot_internals=True);

### A more realistic test

This decoding test uses an audio file played over a cell phone loudspeaker and recorded with a laptop microphone; you can clearly hear the background noise:

In [ ]:
from scipy.io import wavfile
f, audio = wavfile.read('out.wav')
audio = audio / max(abs(audio))
assert f==32000, 'Receiver works at a sampling rate of 32KHz'
IPython.display.Audio(audio, rate=Fs)

The loudpseaker, the microphone and the audio circuitry also affects the signal; here you can see how the phase reversal in the pilot becomes "smeared" over time:

In [ ]:
plt.plot(audio[18500:19500]);

As a consequence, there will be several false positives in the phase reversal detection process. By keeping the result with the largest energy value we are able to achieve synchronization.

In [ ]:
m = Receiver().receive(audio, plot_internals=True);

In particular, note how much more difficult it is to detect the phase reversal over an acoustic channel:

In [ ]:
plt.plot((m['pilot'][18800:19100]))
plt.plot(m['timing'][18800:19100]);